In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 1


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 08-05 16:04:13] ax.service.ax_client: Generated new trial 16 with parameters {'Drug': 94, 'SL_1': 0, 'SL_2': 0, 'SL_3': 56, 'LL_1': 81, 'LL_2': 55, 'LL_3': 95, 'P_1': 0, 'P_2': 12, 'P_3': 0, 'S_1': 41, 'S_2': 0, 'S_3': 27, 'Water': 0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude t

18 min


In [7]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,41,0,27,0
1,29,0,38,53
2,100,0,31,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [8]:
mask = (unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']] == 0).all(axis=1)

# print the rows that are about to change
print("Rows to be updated (before):")
print(unlabeled_trial.loc[mask])

# set Water to 100 on those rows
unlabeled_trial.loc[mask, 'Water'] = 100

# print the rows after the update
print("\nRows after update:")
print(unlabeled_trial.loc[mask])

Rows to be updated (before):
    trial_index  Drug  SL_1  SL_2  SL_3  LL_1  LL_2  LL_3  P_1  P_2  ...  S_1  \
3            19    94     0     0    47    60   100    82    0    0  ...    0   
4            20    96     0     0    46    59     0    95    0    0  ...    0   
5            21    97     0     0    65    91     0    87    0    0  ...    0   
6            22    97     0     0    69    89     0   100    0    0  ...    0   
7            23    97     0     0    65    87     0    99    0    0  ...    0   
8            24    97     0     0    67    86     0    97    0    0  ...    0   
9            25    96     0     0    67    85     0   100    0    0  ...    0   
10           26    97     0     0    65    90     0    90    0    0  ...    0   
11           27    96     0     0    41    50     0    93    0    0  ...    0   
12           28    98     0     0    70    90     0   100    0    0  ...    0   
13           29    97     0     0    69    86     0   100    0    0  ...    0   

In [9]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,41,0,27,0
1,29,0,38,53
2,100,0,31,0
3,0,0,0,100
4,0,0,0,100
5,0,0,0,100
6,0,0,0,100
7,0,0,0,100
8,0,0,0,100
9,0,0,0,100


In [10]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.0,16.0,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.0,16.000000,16.0,16.000000,16.000000
mean,23.500000,0.285687,0.0,0.0,0.166187,0.220375,0.049188,0.276563,0.0,0.001937,0.0,0.100500,0.0,0.059437,0.840125
std,4.760952,0.029570,0.0,0.0,0.050010,0.067172,0.114893,0.028994,0.0,0.007750,0.0,0.237029,0.0,0.131083,0.356128
min,16.000000,0.239000,0.0,0.0,0.000000,0.000000,0.000000,0.214000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000
25%,19.750000,0.273750,0.0,0.0,0.152750,0.204250,0.000000,0.261250,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.000000
50%,23.500000,0.279500,0.0,0.0,0.189500,0.246500,0.000000,0.281000,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.000000
75%,27.250000,0.286500,0.0,0.0,0.193250,0.251000,0.000000,0.284750,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.000000
max,31.000000,0.347000,0.0,0.0,0.196000,0.270000,0.386000,0.332000,0.0,0.031000,0.0,0.763000,0.0,0.397000,1.000000


In [11]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.0,16.0,16.000000,16.000000,16.00000,16.00000,16.0,16.00,16.0,16.00000,16.0,16.000000,16.000000
mean,23.500000,95.625000,0.0,0.0,57.187500,75.875000,15.75000,93.00000,0.0,0.75,0.0,10.62500,0.0,6.000000,84.562500
std,4.760952,2.680174,0.0,0.0,17.807185,23.944032,35.08561,9.29516,0.0,3.00,0.0,26.72795,0.0,13.058841,35.019935
min,16.000000,87.000000,0.0,0.0,0.000000,0.000000,0.00000,67.00000,0.0,0.00,0.0,0.00000,0.0,0.000000,0.000000
25%,19.750000,95.500000,0.0,0.0,53.750000,75.750000,0.00000,89.25000,0.0,0.00,0.0,0.00000,0.0,0.000000,100.000000
50%,23.500000,97.000000,0.0,0.0,65.000000,86.000000,0.00000,96.00000,0.0,0.00,0.0,0.00000,0.0,0.000000,100.000000
75%,27.250000,97.000000,0.0,0.0,67.500000,89.250000,0.00000,100.00000,0.0,0.00,0.0,0.00000,0.0,0.000000,100.000000
max,31.000000,98.000000,0.0,0.0,70.000000,91.000000,100.00000,100.00000,0.0,12.00,0.0,100.00000,0.0,38.000000,100.000000


In [12]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.0,16.0,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.0,16.00000,16.0,16.000000,16.000000,16.000000
mean,23.500000,99.990625,0.0,0.0,58.165625,77.131250,17.215625,96.796875,0.0,0.678125,0.0,100.50000,0.0,59.437500,840.125000,0.021875
std,4.760952,10.349367,0.0,0.0,17.503368,23.510302,40.212592,10.147963,0.0,2.712500,0.0,237.02855,0.0,131.083164,356.127669,0.154886
min,16.000000,83.650000,0.0,0.0,0.000000,0.000000,0.000000,74.900000,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.000000,-0.350000
25%,19.750000,95.812500,0.0,0.0,53.462500,71.487500,0.000000,91.437500,0.0,0.000000,0.0,0.00000,0.0,0.000000,1000.000000,0.000000
50%,23.500000,97.825000,0.0,0.0,66.325000,86.275000,0.000000,98.350000,0.0,0.000000,0.0,0.00000,0.0,0.000000,1000.000000,0.000000
75%,27.250000,100.275000,0.0,0.0,67.637500,87.850000,0.000000,99.662500,0.0,0.000000,0.0,0.00000,0.0,0.000000,1000.000000,0.000000
max,31.000000,121.450000,0.0,0.0,68.600000,94.500000,135.100000,116.200000,0.0,10.850000,0.0,763.00000,0.0,397.000000,1000.000000,0.350000


In [13]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug        1.6
SL_1        0.0
SL_2        0.0
SL_3        0.9
LL_1        1.2
LL_2        0.3
LL_3        1.5
P_1         0.0
P_2         0.0
P_3         0.0
S_1         1.6
S_2         0.0
S_3         1.0
Water      13.4
Solvent     0.0
dtype: float64

In [14]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [15]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [16]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [17]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [18]:
updated_script

('protocol/iteration_1_OT_2_protocol_1.py',
 'protocol/iteration_1_OT_2_protocol_2.py')

# Perform experiment

# Results

In [19]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,16,0.239,0,0,0.142,0.206,0.140,0.242,0,0.031,0,0.603,0,0.397,0.000,7,0
1,17,0.347,0,0,0.000,0.000,0.386,0.267,0,0.000,0,0.242,0,0.317,0.442,4,0
2,18,0.282,0,0,0.170,0.245,0.000,0.303,0,0.000,0,0.763,0,0.237,0.000,5,0
3,19,0.245,0,0,0.123,0.157,0.261,0.214,0,0.000,0,0.000,0,0.000,1.000,4,0
4,20,0.324,0,0,0.155,0.199,0.000,0.321,0,0.000,0,0.000,0,0.000,1.000,3,0
5,21,0.285,0,0,0.191,0.268,0.000,0.256,0,0.000,0,0.000,0,0.000,1.000,3,0
6,22,0.273,0,0,0.194,0.251,0.000,0.282,0,0.000,0,0.000,0,0.000,1.000,3,0
7,23,0.279,0,0,0.187,0.250,0.000,0.284,0,0.000,0,0.000,0,0.000,1.000,3,0
8,24,0.280,0,0,0.193,0.248,0.000,0.280,0,0.000,0,0.000,0,0.000,1.000,3,0
9,25,0.276,0,0,0.193,0.244,0.000,0.287,0,0.000,0,0.000,0,0.000,1.000,3,0


In [20]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Caution,A1,308.42311175789376,0.57079632322136808
1,Caution,A2,285.88305775370759,0.57079632496086807
2,Caution,A3,--,--
3,Caution,A4,270.92583207758651,0.57079628593814202
4,Good,A5,273.29468132767141,0.43386434651604239
5,Caution,A6,290.72683601466616,0.51959509378172386
6,Caution,B1,1837.6373697426225,0.57079632468978181
7,Caution,B2,3009.4996453331655,0.57079632397485103
8,Caution,B3,1010.4678648592004,0.57079631447446877
9,Caution,B4,273.06463574745334,0.57079632443069084


In [21]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,1000,0,1,0,0
1,Formulation 2,1000,0,1,0,0
2,Formulation 3,1000,0,1,0,0
3,Formulation 4,1000,0,1,0,0
4,Formulation 5,1000,0,1,0,0
5,Formulation 6,1000,0,1,0,0
6,Formulation 7,1000,0,1,0,0
7,Formulation 8,1000,0,1,0,0
8,Formulation 9,1000,0,1,0,0
9,Formulation 10,1000,0,1,0,0


In [22]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,16,0.239,0,0,0.142,0.206,0.140,0.242,0,0.031,...,0.397,0.000,7,0,Formulation 1,1000,0,1,0,0
1,17,0.347,0,0,0.000,0.000,0.386,0.267,0,0.000,...,0.317,0.442,4,0,Formulation 2,1000,0,1,0,0
2,18,0.282,0,0,0.170,0.245,0.000,0.303,0,0.000,...,0.237,0.000,5,0,Formulation 3,1000,0,1,0,0
3,19,0.245,0,0,0.123,0.157,0.261,0.214,0,0.000,...,0.000,1.000,4,0,Formulation 4,1000,0,1,0,0
4,20,0.324,0,0,0.155,0.199,0.000,0.321,0,0.000,...,0.000,1.000,3,0,Formulation 5,1000,0,1,0,0
5,21,0.285,0,0,0.191,0.268,0.000,0.256,0,0.000,...,0.000,1.000,3,0,Formulation 6,1000,0,1,0,0
6,22,0.273,0,0,0.194,0.251,0.000,0.282,0,0.000,...,0.000,1.000,3,0,Formulation 7,1000,0,1,0,0
7,23,0.279,0,0,0.187,0.250,0.000,0.284,0,0.000,...,0.000,1.000,3,0,Formulation 8,1000,0,1,0,0
8,24,0.280,0,0,0.193,0.248,0.000,0.280,0,0.000,...,0.000,1.000,3,0,Formulation 9,1000,0,1,0,0
9,25,0.276,0,0,0.193,0.244,0.000,0.287,0,0.000,...,0.000,1.000,3,0,Formulation 10,1000,0,1,0,0


In [23]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,16,0.239,0,0,0.142,0.206,0.140,0.242,0,0.031,...,7,0,Formulation 1,1000,0,1,0,0,0.0,0
1,17,0.347,0,0,0.000,0.000,0.386,0.267,0,0.000,...,4,0,Formulation 2,1000,0,1,0,0,0.0,0
2,18,0.282,0,0,0.170,0.245,0.000,0.303,0,0.000,...,5,0,Formulation 3,1000,0,1,0,0,0.0,0
3,19,0.245,0,0,0.123,0.157,0.261,0.214,0,0.000,...,4,0,Formulation 4,1000,0,1,0,0,0.0,0
4,20,0.324,0,0,0.155,0.199,0.000,0.321,0,0.000,...,3,0,Formulation 5,1000,0,1,0,0,0.0,0
5,21,0.285,0,0,0.191,0.268,0.000,0.256,0,0.000,...,3,0,Formulation 6,1000,0,1,0,0,0.0,0
6,22,0.273,0,0,0.194,0.251,0.000,0.282,0,0.000,...,3,0,Formulation 7,1000,0,1,0,0,0.0,0
7,23,0.279,0,0,0.187,0.250,0.000,0.284,0,0.000,...,3,0,Formulation 8,1000,0,1,0,0,0.0,0
8,24,0.280,0,0,0.193,0.248,0.000,0.280,0,0.000,...,3,0,Formulation 9,1000,0,1,0,0,0.0,0
9,25,0.276,0,0,0.193,0.244,0.000,0.287,0,0.000,...,3,0,Formulation 10,1000,0,1,0,0,0.0,0


In [24]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1,0.0,0.0,0.0,1,0,0.583333
1,0,1,0.0,0.0,0.0,1,0,0.333333
2,0,1,0.0,0.0,0.0,1,0,0.416667
3,0,1,0.0,0.0,0.0,1,0,0.333333
4,0,1,0.0,0.0,0.0,1,0,0.250000
5,0,1,0.0,0.0,0.0,1,0,0.250000
6,0,1,0.0,0.0,0.0,1,0,0.250000
7,0,1,0.0,0.0,0.0,1,0,0.250000
8,0,1,0.0,0.0,0.0,1,0,0.250000
9,0,1,0.0,0.0,0.0,1,0,0.250000


In [25]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1,0.0,0.0,0.0,1,0,0.583333
1,0,1,0.0,0.0,0.0,1,0,0.333333
2,0,1,0.0,0.0,0.0,1,0,0.416667
3,0,1,0.0,0.0,0.0,1,0,0.333333
4,0,1,0.0,0.0,0.0,1,0,0.250000
5,0,1,0.0,0.0,0.0,1,0,0.250000
6,0,1,0.0,0.0,0.0,1,0,0.250000
7,0,1,0.0,0.0,0.0,1,0,0.250000
8,0,1,0.0,0.0,0.0,1,0,0.250000
9,0,1,0.0,0.0,0.0,1,0,0.250000


In [26]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [27]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [28]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,16,0.239,0,0,0.142,0.206,0.140,0.242,0,0.031,...,0.583333,0,Formulation 1,1,0,1,0,0,0,0
1,17,0.347,0,0,0.000,0.000,0.386,0.267,0,0.000,...,0.333333,0,Formulation 2,1,0,1,0,0,0,0
2,18,0.282,0,0,0.170,0.245,0.000,0.303,0,0.000,...,0.416667,0,Formulation 3,1,0,1,0,0,0,0
3,19,0.245,0,0,0.123,0.157,0.261,0.214,0,0.000,...,0.333333,0,Formulation 4,1,0,1,0,0,0,0
4,20,0.324,0,0,0.155,0.199,0.000,0.321,0,0.000,...,0.250000,0,Formulation 5,1,0,1,0,0,0,0


In [29]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [30]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [31]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-06 06:24:45] ax.service.ax_client: Completed trial 16 with data: {'Solu': (0, 0), 'Size': (1, 0), 'PDI': (1, 0)}.
[INFO 08-06 06:24:45] ax.service.ax_client: Completed trial 17 with data: {'Solu': (0, 0), 'Size': (1, 0), 'PDI': (1, 0)}.
[INFO 08-06 06:24:45] ax.service.ax_client: Completed trial 18 with data: {'Solu': (0, 0), 'Size': (1, 0), 'PDI': (1, 0)}.
[INFO 08-06 06:24:45] ax.service.ax_client: Completed trial 19 with data: {'Solu': (0, 0), 'Size': (1, 0), 'PDI': (1, 0)}.
[INFO 08-06 06:24:45] ax.service.ax_client: Completed trial 20 with data: {'Solu': (0, 0), 'Size': (1, 0), 'PDI': (1, 0)}.
[INFO 08-06 06:24:45] ax.service.ax_client: Completed trial 21 with data: {'Solu': (0, 0), 'Size': (1, 0), 'PDI': (1, 0)}.
[INFO 08-06 06:24:45] ax.service.ax_client: Completed trial 22 with data: {'Solu': (0, 0), 'Size': (1, 0), 'PDI': (1, 0)}.
[INFO 08-06 06:24:45] ax.service.ax_client: Completed trial 23 with data: {'Solu': (0, 0), 'Size': (1, 0), 'PDI': (1, 0)}.
[INFO 08-06 06:2